<a href="https://colab.research.google.com/github/DGkurnia/discuss1/blob/main/airquality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyek Analisis Data: [Input Nama Dataset]
- **Nama:** [Derfansyah Guswiranata Kurnia]
- **Email:** [weerakurnia@gmail.com]
- **ID Dicoding:** [dgkurnia]

## Menentukan Pertanyaan Bisnis

- Pertanyaan 1 : Bagaimana kondisi suhu di dua belas kota dari tahun 2013 samapai 2017 ?
- Pertanyaan 2 : Bagaimana pemeriksaan ukuran partikuler (untuk PM 2.5 dan PM 10) di dua belas kota dari tahun 2013 sampai 2017 ?.
- Pertanyaan 3 : Bagaimana pemeriksaan kadar sulfur dioksida (SO2), kadar nitrogen monoksida (NO) dan karbon monoksida (CO) di dua belas kota dari tahun 2013 sampai 2017 ?.
- Pertanyaan 4 : Bagaimana pemeriksaan kondisi gas ozone (O3) dari tahun 2013 sampai 2017 ?.

In [1]:
from google.colab import drive
drive.mount('/content/drive') #dasar instalasi file dari Google Drive

Mounted at /content/drive


## Import Semua Packages/Library yang Digunakan

In [2]:
#persiapan perpustakaan dan file untuk pembersihan
import numpy as np
import pandas as pd #dasar pengelolaan data
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime # dasar pengelolaan data berbasis waktu
from scipy.interpolate import interp1d

## Data Wrangling

In [3]:
#persiapan insersi data
dfaotizhongxin = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/aotizhongxin.csv')
dfchangping = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/changping.csv')
dfdingling = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/dingling.csv')
dfdongsi = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/dongsi.csv')
dfguanyuan = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/guanyuan.csv')
dfgucheng = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/gucheng.csv')
dfhuairou = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/huairo.csv')
dfnonzhanguan = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/nonzhanguan.csv')
dfshunyi = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/shunyi.csv')
dftiantan = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/tiantan.csv')
dfwanliu = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/wanliu.csv')
dfwanshouxigong = pd.read_csv('/content/drive/MyDrive/csv-files/aqins/wanshouxigong.csv')
"""
Kota awal adalah Aotizhungxin dan kota akhir adalah Wanshouxigong
"""

'\nKota awal adalah Aotizhungxin dan kota akhir adalah Wanshouxigong\n'

### Tahap Pengumpulan (Gathering Data)
#### rincian dari pengumpulan data : ada 12 dataset kota yang terdiri dari
* Nomor
* Tahun
* Bulan
* Jam
* kadar partikulat (PM 2.5 dan 10)
* kadar senyawa SO2, NO, CO, dan O3 (dalam mikrogram/ m^3)
* tempratur (dalam celsius)
* Nilai tekanan (dalam hektoPascal)
* arah angin dan kecepatan angin
-- Nilai Tahun, bulan dan jam bisa diubah menjadi tanggal.


### Assessing Data
#### Hal yang diperhatikan dalam pemeriksaan kadar udara
- Total data duplikat
-- Aotizhongxin : tidak ada
-- Changpin : Tidak ada
- Jumlah data hilang di masing-masing unit
-- Kota Aotizhingxin : 925 data di PM2.5, 718 di PM 10. Tiga kolom memiliki data kosong di atas 500
-- Kota Changpin : banyak data hilang berada di pemeriksaan empat gas tersebut
--- Kota lain memiliki kecendrungan untuk memiliki data kosong di kondisi tersebut
#### Hal unik lain
Forrmat tanggal yang salah (awalnya tahun, bulan, hari, dan jam) yang dipisah

In [4]:
"""
pemeriksaan data dari data kurang lengkap untuk enam kota pertama :
(kota Aotizhongxin sampai Huairo)
"""
#dfaotizhongxin.info()
#dfaotizhongxin.isnull().sum()
#dfaotizhongxin.duplicated().sum() # kota Aotizhongxin
#dfchangping.info()
#dfchangping.isnull().sum()
#dfchangping.duplicated().sum() # Changping
#dfdongsi.isnull().sum()
#dfdongsi.duplicated().sum()
#dfdingling.info()
#dfdingling.isnull().sum()
#dfdingling.duplicated().sum() #Dingling

'\npemeriksaan data dari data kurang lengkap untuk enam kota pertama :\n(kota Aotizhongxin sampai Huairo)\n'

In [ ]:
"""
pemeriksaan data dari data kurang lengkap untuk enam kota terakhir :
(kota Nonzhanguan sampai Wanshouxigong)
"""
#dfnonzhanguan.info()
#dfnonzhanguan.isnull().sum()
#dfnonzhanguan.duplicated().sum() #Nonzhanguan
#dfwanshouxigong.info()

'\npemeriksaan data dari data kurang lengkap untuk enam kota terakhir :\n(kota Nonzhanguan sampai Wanshouxigong)\n'

### Cleaning Data
#### Pengantar : Data ini bisa dimanfaatkan untuk pengaturan sehingga proses yang sesuai untuk pembersihan data adalah interpolasi kolom indikator yang berangka demi menjaga aspek kolom yang hilang. Di sisilain, pengisian data dengan modus berlaku untuk kolom non integer
Tahapan pembersihan data
(hal ini berlaku untuk 12 kota)
1.   Pengubahan format waktu dan menghapus format salah
2. Proses interpolasi untuk kolom hilang di indikator
3. Pengisian nilai non numerik yang hilang




In [5]:
#pembersihan bagian pertama (pengubahan format data waktu) untuk dua belas kota
dfaotizhongxin['datetime'] = dfaotizhongxin[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfchangping['datetime'] = dfchangping[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfdingling['datetime'] = dfdingling[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfdongsi['datetime'] = dfdongsi[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfguanyuan['datetime'] = dfguanyuan[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfgucheng['datetime'] = dfgucheng[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfhuairou['datetime'] = dfhuairou[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfnonzhanguan['datetime'] = dfnonzhanguan[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfshunyi['datetime'] = dfshunyi[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dftiantan['datetime'] = dftiantan[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfwanliu['datetime'] = dfwanliu[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)
dfwanshouxigong['datetime'] = dfwanshouxigong[['year', 'month', 'day', 'hour']].apply(lambda x: datetime(*x), axis=1)

#penghapusan kolom 'tahun' dan tiga indeks lain untuk 12 kota (dari Aotizhongxin sampai Wanshouxigong)

dfaotizhongxin.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfchangping.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfdingling.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfdongsi.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfguanyuan.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfgucheng.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfhuairou.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfnonzhanguan.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfshunyi.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dftiantan.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfwanliu.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)
dfwanshouxigong.drop(['year', 'month', 'day', 'hour'], axis=1, inplace=True)

#pemeriksaan kolom untuk memeriksa jika kota sudah memiliki indikator waktu
#dfaotizhongxin.head(10) #Kota Aotizhongxin
#dfchangping.head(10)# Kota Changping
#dfdingling.head(10)# Kota Dingling
#dfdongsi.head(10)# Kota Dongsi
#dfguanyuan.head(10)# Kota Guanyuan
#dfgucheng.head(10)# Kota Gucheng
#dfhuairou.head(10)# Kota Huairou
#dfnonzhanguan.head(10) #kota Nanzhanguan
#dfshunyi.head(10) #kota sunyi
#dftiantan.head(10) #Kota Tiantan
#dfwanliu.head(10) #Kota Wanliu
#dfwanshouxigong.head(10) #Kota Wanshouxigong
#hasil ini untuk memeriksa penghapusan kolom waktu

In [6]:
"""
#bagian dua (proses interpolasi kolom)
Tahap pertama pemeriksaan data yang hilang untuk masing-masing kota dari Aotizhongxing sampai Wanshouxigong
"""
# pemeriksan total kolom kosong untuk interpolasi
#dfaotizhongxin.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Aotizhongxing)
#dfchangping.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Changping)
#dfdingling.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Dingling)
#dfdongsi.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Dongsi)
#dfguanyuan.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Guanyuan)
#dfgucheng.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Gucheng)
#dfhuairou.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Huairou)
#dfnonzhanguan.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Nonzhanguan)
#dfshunyi.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Shunyi)
#dftiantan.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Tiantan)
#dfwanliu.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Wanliu)
#dfwanshouxigong.isnull().sum() #untuk memeriksa kolom sebelum interpolasi (Wanshouxigong)

'\n#bagian dua (proses interpolasi kolom)\nTahap pertama pemeriksaan data yang hilang untuk masing-masing kota dari Aotizhongxing sampai Wanshouxigong\n'

##Resume Kolom Hilang Data untuk Interpolasi:

*   Sebagian besar kolom hilang bisa 'diinterpolasi'
*   Kota



In [7]:
# persiapan interpolasi (Aotizhongxin sampai terakhir) di kolom PM2.5 dan PM10 sebelum pembersihan data
dfaotizhongxin['PM2.5'] = dfaotizhongxin['PM2.5'].interpolate(method='linear')
dfaotizhongxin['PM10'] = dfaotizhongxin['PM10'].interpolate(method='linear')
#Area Aotizhongxin
dfchangping['PM2.5'] = dfchangping['PM2.5'].interpolate(method='linear')
dfchangping['PM10'] = dfchangping['PM10'].interpolate(method='linear')
#Area Changping
dfdingling['PM2.5'] = dfdingling['PM2.5'].interpolate(method='linear')
dfdingling['PM10'] = dfdingling['PM10'].interpolate(method='linear')
#Area Dingling
dfdongsi['PM2.5'] = dfdongsi['PM2.5'].interpolate(method='linear')
dfdongsi['PM10'] = dfdongsi['PM10'].interpolate(method='linear')
#Area Dongsi
dfguanyuan['PM2.5'] = dfguanyuan['PM2.5'].interpolate(method='linear')
dfguanyuan['PM10'] = dfguanyuan['PM10'].interpolate(method='linear')
#Area Guanyuan
dfgucheng['PM2.5'] = dfgucheng['PM2.5'].interpolate(method='linear')
dfgucheng['PM10'] = dfgucheng['PM10'].interpolate(method='linear')
#Area Gucheng
dfhuairou['PM2.5'] = dfhuairou['PM2.5'].interpolate(method='linear')
dfhuairou['PM10'] = dfhuairou['PM10'].interpolate(method='linear')
#Area Huairou
dfnonzhanguan['PM2.5'] = dfnonzhanguan['PM2.5'].interpolate(method='linear')
dfnonzhanguan['PM10'] = dfnonzhanguan['PM10'].interpolate(method='linear')
#Area Nonzhanguan

In [8]:
#pemeriksaan nilai interpolasi
dfaotizhongxin[['PM2.5','PM10']] #untuk memeriksa kolom setelah interpolasi (Aotizhongxing)
dfchangping[['PM2.5','PM10']] #untuk memeriksa kolom setelah interpolasi (Changping)
dfdingling[['PM2.5','PM10']] #untuk memeriksa kolom setelah interpolasi (Dingling)
dfdongsi[['PM2.5','PM10']] #untuk memeriksa kolom setelah interpolasi (Dongsi)
#pemeriksaan hasil interpolasi
dfaotizhongxin.isnull().sum() #untuk memeriksa kolom setelah interpolasi (Aotizhongxing)

,0
No,0
year,0
month,0
day,0
hour,0
PM2.5,0
PM10,0
SO2,935
NO2,1023
CO,1776


## Exploratory Data Analysis (EDA)

**Insight:**
- xxx
- xxx

**Insight:**
- xxx
- xxx

**Insight:**
- xxx
- xxx

### Explore The insight

**Insight:**
- xxx
- xxx

## Visualization & Explanatory Analysis

### Pertanyaan 1:

### Pertanyaan 2:

**Insight:**
- xxx
- xxx

## Analisis Lanjutan (Opsional)

## Conclusion

- Conclution pertanyaan 1
- Conclution pertanyaan 2